In [1]:
import sys
import torch
sys.path.append('/home/zengdun/FedLab/')
import torchvision
import torchvision.transforms as transforms
from copy import deepcopy
import random
import os 

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [ ]:
# unit test of `SerialHandler`
from fedlab_utils.dataset.sampler import DistributedSampler
from fedlab_core.client.serial_handler import SerialHandler
from models.lenet import LeNet
from fedlab_utils.aggregator import Aggregators
from fedlab_utils.serialization import SerializationTool
from fedlab_utils.functional import evaluate

model = LeNet().cuda()
aggregator = Aggregators.fedavg_aggregate
criterion = torch.nn.CrossEntropyLoss()

trainset = torchvision.datasets.MNIST(
    root='/home/zengdun/datasets/mnist/', train=True, download=True, transform=transforms.ToTensor())

testset = torchvision.datasets.MNIST(
    root='/home/zengdun/datasets/mnist/', train=False, download=True, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(testset, batch_size=len(testset), drop_last=False, shuffle=False)

sim_client_num = 100  # client总数

test_handler = SerialHandler(local_model=deepcopy(model), aggregator=aggregator, dataset=trainset, sim_client_num=sim_client_num)

model_params = SerializationTool.serialize_model(model)

to_select = [i+1 for i in range(sim_client_num)] # client_id 从1开始

for global_epoch in range(2):
    selection = random.sample(to_select, 10)
    print(selection)
    model_params = test_handler.train(epochs=10, batch_size=128, idx_list=selection, model_parameters=model_params, cuda=True)
    SerializationTool.deserialize_model(model, model_params)
    evaluate(model, criterion, test_loader, cuda=True)


In [2]:
# sampler test
from  fedlab_utils.dataset.slicing import noniid_slicing
from fedlab_utils.dataset.sampler import AssignSampler
trainset = torchvision.datasets.MNIST(
    root='/home/zengdun/datasets/mnist/', train=True, download=True, transform=transforms.ToTensor())

dict = noniid_slicing(trainset, 100, 200)
sampler = AssignSampler(dict[0], shuffle=True)
train_loader = torch.utils.data.DataLoader(trainset, sampler = sampler, batch_size=100)
for data, label in train_loader:
    print(label)

tensor([4, 4, 0, 0, 4, 4, 0, 0, 0, 4, 4, 4, 0, 0, 4, 0, 4, 0, 0, 0, 4, 4, 4, 4,
        0, 4, 0, 4, 4, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 4, 0, 0,
        4, 4, 4, 4, 0, 4, 4, 4, 0, 0, 0, 4, 4, 4, 4, 4, 4, 0, 0, 4, 0, 0, 0, 0,
        0, 4, 0, 4, 4, 4, 4, 0, 4, 0, 0, 0, 4, 0, 4, 0, 4, 0, 4, 4, 4, 4, 4, 0,
        4, 0, 4, 0])
tensor([4, 0, 4, 0, 4, 4, 0, 4, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 0, 0, 0, 4, 4,
        0, 0, 4, 4, 4, 4, 0, 0, 4, 0, 0, 4, 4, 0, 4, 4, 4, 4, 4, 0, 4, 0, 4, 0,
        0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 4, 4, 4, 0, 4, 0, 4, 0, 0, 0, 4, 0, 0, 4,
        4, 4, 4, 0, 4, 0, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 0, 0, 4, 0, 0, 4, 0, 4,
        4, 0, 0, 0])
tensor([4, 0, 4, 4, 4, 4, 0, 0, 0, 0, 4, 4, 0, 0, 0, 4, 0, 0, 4, 4, 4, 4, 4, 0,
        0, 4, 0, 4, 4, 0, 0, 0, 4, 0, 4, 0, 0, 4, 4, 4, 4, 0, 0, 4, 0, 0, 4, 4,
        0, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 4, 0, 0, 4, 0, 4, 4,
        4, 0, 0, 4, 0, 0, 4, 0, 4, 4, 0, 4, 4, 4, 0, 4, 0, 4, 4, 0, 0, 0, 0, 0